In [ ]:
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using Revise
using HybridRobotDynamics
using HybridTrajIpopt
using ImpactSafety

In [ ]:
# Define hopper model
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using Revise
using HybridRobotDynamics
using HybridTrajIpopt

# Define hopper model
system = hopper()

# Define stage and terminal cost functions
Q = 1e0 * diagm([1.0; 1e1; 1.0; 1e1; 1e-1*ones(4)])
R = 1e-8 * Matrix(I(system.nu))
Qf = 2e0 * Q
stage = (x,u) -> x'*Q*x + u'*R*u
terminal = x -> x'*Qf*x

# Define trajopt parameters
N = 20
Δt = 0.05
hs = ImplicitIntegrator(:hermite_simpson)
params = ProblemParameters(
    system, hs, stage, terminal, N;
    Δt=Δt
)

# Define transition sequence and terminal guard
impact = system.transitions[:impact]
liftoff = system.transitions[:liftoff]
sequence = [
    TransitionTiming(3, liftoff),
    TransitionTiming(8, impact),
    TransitionTiming(13, liftoff),
    TransitionTiming(18, impact)
]
term_guard = impact.guard

# Define reference trajectory and initial conditions
xic = [0.0; 1.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0]
xgc = [5.0; 5.0; 5.0; 4.0; 0.0; 0.0; 0.0; 0.0]
xrefs = repeat(xgc, N)
urefs = zeros((N-1) * system.nu)

# Initial guess
us = urefs
rk4 = ExplicitIntegrator(:rk4)
xs = roll_out(system, rk4, N, Δt, us, xic, :stance)
plot_2d_states(
    N, system.nx, (3,4), xrefs;
    title = "Initial Guess",
)

nothing

In [ ]:
# Solve using Ipopt
y0 = compose_trajectory(params.dims, params.idx, xs, us)
cb = SolverCallbacks(
    params, sequence, term_guard, xrefs, urefs, xic;
    gauss_newton=true
)
sol = ipopt_solve(params, cb, y0; max_iter=1000)

# Visualize
xs, us, Δts = decompose_trajectory(params.idx, sol.x)
plot_2d_states(N, system.nx, (3,4), xs;)

# Init TVLQR policy with RK4 integration
yref = sol.x
rk4 = ExplicitIntegrator(:rk4)
tvlqr = TimeVaryingLQR(params, rk4, Q, R, Qf, sequence, yref)

nothing

In [ ]:
function post_impact_velocity(
    flow::Union{ControlAffineFlow, Function},
    trn::Transition,
    vidx::UnitRange{Int},
    x::Vector,
    u::Vector,
    Δt::Vector
)::Vector
    x1 = x + Δt*flow(x, u)
    xJ = trn.reset(x1)
    return xJ[vidx]
end

function opos(o::Vector)::Vector
    return o[1:2]
end

function ovel(o::Vector)::Vector
    return o[3:4]
end

function xpos(x::Vector)::Vector
    return x[1:2]
end

function xvel(x::Vector)::Vector
    return x[5:6]
end

function decoupled_Φ(n::Real, k::Real, dmin::Real, o::Vector, p::Vector, v::Vector)
    rel_dist = opos(o) - p
    d = norm(rel_dist)
    d_unit = rel_dist / d
    rel_vel = ovel(o) - v
    ḋ = rel_vel' * d_unit
    return dmin*n .- d*n .- k*ḋ
end

function Φ(n::Real, k::Real, dmin::Real, o::Vector, x::Vector)::Vector
    #@show n, k, dmin
    rel_dist = opos(o) - xpos(x)
    d = norm(rel_dist)
    d_unit = rel_dist / d
    rel_vel = ovel(o) - xvel(x)
    ḋ = rel_vel' * d_unit
    return [dmin*n - d*n - k*ḋ]
end

function Φ̇ssa(Φargs::Tuple, η::Real, x::Vector)::Vector
    Φval = Φ(Φargs..., x)
    if Φval[1] > 1e-9
        return [-η]
    elseif Φval[1] > -1e-9
        return [0.0]
    else
        return [Inf]
    end
end

function Φ̇cbf(Φargs::Tuple, λ::Real, x::Vector)::Vector
    Φval = Φ(Φargs..., x)
    return -λ*Φval
end

function guard(x::Vector)
    return x[4]
end

function plastic_impact(x::Vector)::Vector
    return [x[1:3]; 1e-9; x[5:6]; zeros(2)]
end

nothing

In [ ]:
# Safety index params
n = 1
κ = 0.1
dmin = 0.5

# cbf params
λ = 1e3

# Safety filter
ϵ = 0.0
cbf = SafetyFilter(system.nu, ϵ, Φ, Φ̇cbf, system.modes[:flight].flow)

# Obstacle state
o = [5, 5, 2, 0.0, 0.0, 0.0]

nothing

nothing

In [ ]:
# Sim settings
speedup = 10
Δtsim = Δt / speedup
Nsim = 1 + (N-1)*speedup

# Timing mapping from sim to TVLQR
kmap = length(tvlqr.idx.u) / Nsim

# Initial conditions
xs_sim = [zeros(system.nx) for k = 1:Nsim]
us_sim = [zeros(system.nu) for k = 1:(Nsim-1)]
xs_sim[1] = xic
mI = system.modes[:stance]

# Simulate
for k = 1 : (Nsim-1)
    # Nominal controller
    klqr = trunc(Int, 1 + k*kmap)
    unom = tvlqr(xs_sim[k], klqr)

    # Filtered controller
    Φargs = (n, κ, dmin, o)
    Φ̇args = (Φargs, λ)
    us_sim[k] = cbf(Φargs, Φ̇args, xs_sim[k], unom; verbose=false)

    Φval = Φ(Φargs..., xs_sim[k])
    if Φval[1] > 0.0
        @show xs_sim[k][1], Φval[1]
    end

    # Integrate smooth dynamics
    xs_sim[k+1] = rk4(mI.flow, xs_sim[k], us_sim[k], Δtsim)
    @show xs_sim[k+1]

    # Reset and update mode if guard is hit
    for (trans, mJ) in mI.transitions
        if trans.guard(xs_sim[k+1]) < 0.0
            xs_sim[k+1] = trans.reset(xs_sim[k+1])
            mI = mJ
            break
        end
    end
end

# Visualize
plot_2d_states(Nsim, system.nx, (3,4), vcat(xs_sim...), xlim=(-10,10), ylim=(-10,10))

nothing